# Set up

## Imports

In [ ]:
# local installs
# !pip install dill
# !pip install xgboost
# !pip install sklearn==0.0.post2

In [102]:
gdrive = False;
path_head = ""

if(gdrive):
  from google.colab import drive
  drive.mount('/content/drive')
  path_head = '/content/drive/MyDrive/Cis523/Bounty/'
  !pip install dill
  !pip install xgboost
  !pip install scikit-learn==1.2.1

In [103]:
# Imports
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import *
import dill as pkl
from xgboost.sklearn import XGBRegressor
import xgboost as xg

import math
from sklearn.metrics import mean_squared_error

In [104]:
print(sk.__version__)
print(xg.__version__)

1.2.1
1.7.5


In [105]:
# load data and make subsets 
x_train = pd.read_csv(path_head + 'training_data.csv') 
y_train = np.genfromtxt(path_head + 'training_labels.csv', delimiter=',', dtype = float).reshape(340134,1)

x_train_subset, x_val, y_train_subset, y_val = sk.model_selection.train_test_split(x_train, y_train, test_size = .15, random_state = 42)
# PDL = PointerDecisionList(base_clf, x_train_subset, y_train_subset, x_val, y_val, 1, 1)

In [106]:
# Import current info
# Note, may not be up to date with global git
global_preds_path = path_head + 'models/global_model/training_predictions.csv'
global_preds = pd.read_csv(global_preds_path, header=None).transpose()
PAS_preds_path = path_head + 'models/PAS/training_predictions.csv'
PAS_preds = pd.read_csv(PAS_preds_path, header=None).transpose()

In [89]:
_, _, old_train_preds, old_val_preds = sk.model_selection.train_test_split(x_train, global_preds, test_size = .15, random_state = 42)

## Helper Functions

In [91]:
# check improvement on validation data
def check_improvement(old_pred, g, h):
    indices = g(x_val)
    new_pred = h(x_val[indices])
    
    improvement = -100000
    if(len(new_pred)!=0):   
        old_RMSE = math.sqrt(mean_squared_error(y_val[indices], old_val_preds[indices]))
        new_RMSE = math.sqrt(mean_squared_error(y_val[indices], new_pred))
        improvement = old_RMSE-new_RMSE
    print(f"improvement: {improvement}")
    if (improvement>0):
        print("\n IMPROVEMENT \n IMPROVEMENT \n IMPROVEMENT \n")
    return improvement

def check_local_improvement(g,h):
    return check_improvement(PAS_preds, g, h)

def check_global_improvement(g,h):
    return check_improvement(global_preds, g, h)

In [28]:
# def check_improvement(old_pred, g, h):
#     indices = g(x_val)
#     old_pred = old_pred[indices]
#     new_pred = h(x_train[indices])
#     old_RMSE = math.sqrt(mean_squared_error(y_train[indices], old_pred))
#     new_RMSE = math.sqrt(mean_squared_error(y_train[indices], new_pred))
#     # print(f"improvement: {old_RMSE-new_RMSE}")
#     if (old_RMSE-new_RMSE>0):
#         print("\n IMPROVEMENT \n IMPROVEMENT \n IMPROVEMENT \n")
#     return old_RMSE-new_RMSE

# def check_local_improvement(g,h):
#     return check_improvement(PAS_preds, g, h)

# def check_global_improvement(g,h):
#     return check_improvement(global_preds, g, h)

In [92]:
def save_pkls(g,h):
    with open(path_head + 'g.pkl', 'wb') as file:
        pkl.dump(g, file)

    # save hypothesis function to h.pkl
    with open(path_head + 'h.pkl', 'wb') as file:
        pkl.dump(h, file)

In [141]:
def train_basic_h(g):
    clf = sk.tree.DecisionTreeRegressor(max_depth = 7, random_state = 42)

    # find group indices on data
    indices = g(x_train_subset)

    # fit model specifically to group
    clf.fit(x_train_subset[indices], y_train_subset[indices])

    # define hypothesis function as bound clf.predict
    h = clf.predict
    
    return h

In [13]:
def train_XGBRegressor(g):
    clf = XGBRegressor(max_depth = 10, random_state = 42)
    indices = g(x_train_subset)
    clf.fit(x_train_subset[indices], y_train_subset[indices])
    h = clf.predict
    return h

# Starter Code

In order to help minimize start up difficulties, we have provided you with a basic ML workflow for this project, as well as a few possible avenues to explore. 

## Section 1: ML Workflow for Submitting *(g,h)* pairs

### 1.0 Pip Installs and Imports

We will be using a package *dill* which is a variant of *pickle*, but allows a bit more expressive byte code serialization. This package is essential to saving your *(g,h)* pairs!.

In [ ]:
!pip install dill

Here is a non-inclusive list of packages you may find helpful

In [ ]:
# Imports
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import *
import dill as pkl

### 1.1 Download/Load Data

Navigate to the project [webpage](https://declancharrison.github.io/CIS_5230_Bias_Bounty_2023/) and click "Download Training Data". Extract the .zip files in the folder where this notebook is located, then run the cell below.

In [ ]:
x_train = pd.read_csv('training_data.csv') 
y_train = np.genfromtxt('training_labels.csv', delimiter=',', dtype = float)

### 1.2 Define a (g,h) pair

Below is an example of training a Decision Tree Regressor on individuals identified as white from the dataset.

In [ ]:
# define group function
def g(X):
    return X['RAC1P'] == 1

# initialize ML hypothesis class
clf = sk.tree.DecisionTreeRegressor(max_depth = 5, random_state = 42)

# find group indices on data
indices = g(x_train)

# fit model specifically to group
clf.fit(x_train[indices], y_train[indices])

# define hypothesis function as bound clf.predict
h = clf.predict

### 1.3 Save Objects

The following cell will save your group model *g* with filename *g.pkl*, and your hypothesis function *h* with filename *h.pkl*.

In [ ]:
# save group function to g.pkl
with open('g.pkl', 'wb') as file:
    pkl.dump(g, file)

# save hypothesis function to h.pkl
with open('h.pkl', 'wb') as file:
    pkl.dump(h, file)

### 1.4 Upload Models to Google Drive and Submit PR Request with Links

Follow instructions on GitHub Repo to submit a *(g,h)* pair update request!

## Section 2: Reducing Workflow Time Requirements by Creating a Local PDL

As you have probably noticed, submitting a *(g,h)* pair to the GitHub repository can take a long time depending on the current workload of the server. To approximate whether or not an update will be accepted, we have provided you the PDL architecture file and a workflow that will mimic your team's private PDL maintained by the server. 

**NOTE: One major caveat is the validation data this workflow uses is a cut from the training data, meaning you will want to refrain from training on it to prevent overfitting.**

The way we suggest getting around this without losing data efficacy is to train a *(g,h)* pair on the subset of training data that does not include the validation set, and attempt the *(g,h)* pair update on the local PDL. If the pair is rejected, you can continue tuning hyperparameters or searching for new groups. If the pair is accepted, you can retrain a new *(g,h)* pair over ALL the training data, and submit this pair to the server for an update. This will allow you to "squeeze all the juice" from your training data and test potential updates much quicker.  

In [ ]:
### DONT CHANGE THIS CELL ###
from pdl import PointerDecisionList

x_train_subset, x_val, y_train_subset, y_val = sk.model_selection.train_test_split(x_train, y_train, test_size = .15, random_state = 42)
base_clf = sk.tree.DecisionTreeRegressor(max_depth = 1, random_state = 42)
base_clf.fit(x_train_subset, y_train_subset)
PDL = PointerDecisionList(base_clf, x_train_subset, y_train_subset, x_val, y_val, 1, 1)

Train your *(g,h)* pair on the subset of training data below:

In [ ]:
# define group function
def g(X):
    return X['RAC1P'] == 2

# initialize ML hypothesis class
clf = sk.tree.DecisionTreeRegressor(max_depth = 5, random_state = 42)

# find group indices on data
indices = g(x_train_subset)

# fit model specifically to group
clf.fit(x_train_subset[indices], y_train_subset[indices])

# define hypothesis function as bound clf.predict
h = clf.predict

Attempt an update using the following syntax

In [ ]:
update_flag = PDL.update(g, h, x_train_subset, y_train_subset, x_val, y_val)

You can put these two together to train a classifier using the whole training dataset after if it has been accepted:

In [ ]:
# define group function
def g(X):
    return X['RAC1P'] == 1

# initialize ML hypothesis class
clf = sk.tree.DecisionTreeRegressor(max_depth = 10, random_state = 42)

# find group indices on training subset
indices = g(x_train_subset)

# fit model specifically to group subset
clf.fit(x_train_subset[indices], y_train_subset[indices])

# define hypothesis function as bound clf.predict
h = clf.predict

# compute PDL update
update_flag = PDL.update(g, h, x_train_subset, y_train_subset, x_val, y_val)

if update_flag:

    # recompute indices over whole training dataset
    indices = g(x_train)

    # refit classifier to full group
    clf.fit(x_train[indices], y_train[indices])

    # define hypothesis function as bound clf.predict
    h = clf.predict    

Submit *(g,h)* pair to GitHub!

**NOTE: You can save your PDL but it will require that your validation set does not change! Thus, you should not change the random state used to split your training data once you create your PDL**

In [ ]:
# save PDL
PDL.save_model()

# open PDL structure
with open('PDL/model.pkl', 'rb') as file:
    PDL = pkl.load(file)

# reload group/hypothesis functions to PDL
PDL.reload_functions()

# Ahmed's groups

In [116]:
# Supgroups from Original Bias Bounties Paper

# male subgroup
def g_o1(X):
    return X['SEX'] == 1

# female subgroup
def g_o2(X):
    return X['SEX'] == 2

# ages 17-24 and non-white
def g_o3(X):
  return (X['RAC1P'] != 1.0) & (X['AGEP'] >= 17) & (X['AGEP'] <= 24)

# over the age of 30 working without pay in a family business
def g_o4(X):
  return (X['AGEP'] > 30) & (X['COW'] == 8.0)

# self-employed in own not incorporated business, professional practice, or farm
def g_o5(X):
  return X['COW'] == 6.0

# over the age of 62 (retired)
def g_o6(X):
  return (X['AGEP'] > 62) & (X['COW'] == 9.0)

# First-line supervisors of office and administrative support workers
def g_o7(X):
  return X['OCCP'] == 5000

# real estate brokers and sales agents
def g_o8(X):
  return X['OCCP'] == 4920

# real estate brokers and sales agents
def g_o9(X):
  return X['OCCP'] == 4700

# office clerks, general
def g_o10(X):
  return X['OCCP'] == 5860

#paint, coating, and adhesive manufacturing
def g_o11(X):
  return X['OCCP'] == 2270

# those born in California, Mexico, or Southeast Asia generally working as medical technicians
def g_o12(X):
  return (X['OCCP'] == 3401) | (X['ST'] == 6.0)

# accountants/Auditors who work 40 hour work weeks
def g_o13(X):
  return (X['OCCP'] == 800) & (X['WKHP'] >= 40)


In [117]:
# Subgroups that seem to be interesting

# military involvement
def g_m1(X):
  return X['MIL'] != 4.0

# minimal education
def g_m2(X):
  return X['SCHL'] < 16.0

# high education
def g_m3(X):
  return X['SCHL'] > 21.0 

# very little hours worked per week
def g_m4(X):
  return X['WKHP'] < 10 

# less than normal hours worked per week
def g_m5(X):
  return (X['WKHP'] >= 10) & (X['WKHP'] < 30)

# normal hours worked per week
def g_m6(X):
  return (X['WKHP'] >= 30) & (X['WKHP'] < 50)

# high hours worked per week
def g_m7(X):
  return (X['WKHP'] >= 50) & (X['WKHP'] < 80)

# very high hours worked per week
def g_m8(X):
  return (X['WKHP'] >= 80)

# people with disabilities
def g_m9(X):
  return (X['DDRS'] == 1.0) | (X['DEAR'] == 1.0) | (X['DEYE'] == 1.0) | \
  (X['DOUT'] == 1.0) | (X['DRAT'] == 1.0) | (X['DREM'] == 1.0)

# non-married men 
def g_m10(X):
  return (X['MAR'] != 1.0) & (X['SEX'] == 1.0)

# non-married women
def g_m11(X):
  return (X['MAR'] != 1.0) & (X['SEX'] == 2.0)

# software developers
def g_m12(X):
  return X['OCCP'] == 1021.0 

# black females 
def g_m13(X):
  return (X['SEX'] == 2.0) & (X['RAC1P'] == 2.0)

# asian men
def g_m14(X):
  return (X['SEX'] == 1.0) & (X['RAC1P'] == 6.0)

# people who take public transporation / walk / bike
def g_m15(X):
  return (X['JWTRNS'] == 2.0) | (X['JWTRNS'] == 3.0) | (X['JWTRNS'] == 5.0) | \
           (X['JWTRNS'] == 10) | (X['JWTRNS'] == 9.0)

# non white/asian recent mothers
def g_m16(X):
  return (X['FER'] == 1.0) & (X['RAC1P'] != 1.0) & (X['RAC1P'] != 6.0)

# teenagers
def g_m18(X):
  return (X['AGEP'] > 12.0) & (X['AGEP'] <= 19.0)

# young adults
def g_m19(X):
  return (X['AGEP'] > 19) & (X['AGEP'] <= 27.0)

# adults
def g_m20(X):
  return (X['AGEP'] > 27) & (X['AGEP'] <= 55.0)

# old ppl
def g_m21(X):
  return (X['AGEP'] > 55.0)

# managers 
def g_m22(X):
  return X['OCCP'] <= 440.0

# engineers
def g_m23(X):
  return (X['OCCP'] >= 1305.0) & (X['OCCP'] <= 1555)

# teachers
def g_m24(X):
  return (X['OCCP'] >= 2205.0) & (X['OCCP'] <= 2555.0)

# people in medicine
def g_m25(X):
  return (X['OCCP'] >= 3000.0) & (X['OCCP'] <= 3550.0)

# speaks more than english
def g_m27(X):
  return X['LANX'] == 1.0 

# speaks only english
def g_m28(X):
  return X['LANX'] == 2.0

# self employed inc black male 
def g_m29(X):
  return (X['COW'] == 7.0) & (X['RAC1P'] == 2.0) & (X['SEX'] == 1.0)

# self employed inc black female 
def g_m30(X):
  return (X['COW'] == 7.0) & (X['RAC1P'] == 2.0) & (X['SEX'] == 2.0)

# self employed inc asian male 
def g_m31(X):
  return (X['COW'] == 7.0) & (X['RAC1P'] == 6.0) & (X['SEX'] == 1.0)

# self employed inc asian female 
def g_m32(X):
  return (X['COW'] == 7.0) & (X['RAC1P'] == 6.0) & (X['SEX'] == 2.0)

# over 75 and self employed
def g_m33(X):
  return (X['COW'] == 7.0) & (X['AGEP'] >= 75)

# self employed not inc black male 
def g_m34(X):
  return (X['COW'] == 6.0) & (X['RAC1P'] == 2.0) & (X['SEX'] == 1.0)

# self employed not inc black female 
def g_m35(X):
  return (X['COW'] == 6.0) & (X['RAC1P'] == 2.0) & (X['SEX'] == 2.0)

# self employed not inc asian male 
def g_m36(X):
  return (X['COW'] == 6.0) & (X['RAC1P'] == 6.0) & (X['SEX'] == 1.0)

# self employed not inc asian female 
def g_m37(X):
  return (X['COW'] == 6.0) & (X['RAC1P'] == 6.0) & (X['SEX'] == 2.0)

# over 75 and self employed
def g_m38(X):
  return (X['COW'] == 6.0) & (X['AGEP'] >= 75)

# people who live in NY / CALI
def g_m39(X):
  return (X['ST'] == 6.0) | (X['ST'] == 36.0)

# some college
def g_m40(X):
  return (X['SCHL'] > 16.0) & (X['SCHL'] < 21.0)

# bachelor degree
def g_m41(X):
  return X['SCHL'] == 21.0

# self employed and works < 10
def g_m42(X):
  return ((X['COW'] == 6.0) | (X['COW'] == 7.0)) & (X['WKHP'] <= 10)

# self employed and works [11, 20] hours
def g_m43(X):
  return ((X['COW'] == 6.0) | (X['COW'] == 7.0)) & ((X['WKHP'] > 10) & (X['WKHP'] <= 20))

# self employed and works [31, 60] hours
def g_m44(X):
  return ((X['COW'] == 6.0) | (X['COW'] == 7.0)) & ((X['WKHP'] > 30) & (X['WKHP'] <= 60))

# self employed and works > 60 hours
def g_m45(X):
  return ((X['COW'] == 6.0) | (X['COW'] == 7.0)) & (X['WKHP'] > 60)

# private profit and works < 10
def g_m46(X):
  return (X['COW'] == 1.0) & (X['WKHP'] <= 10)

# self employed and works [11, 20] hours
def g_m47(X):
  return (X['COW'] == 1.0) & (X['WKHP'] > 10) & (X['WKHP'] <= 20)

# self employed and works [31, 60] hours
def g_m48(X):
  return (X['COW'] == 1.0) & (X['WKHP'] > 30) & (X['WKHP'] <= 60)

# self employed and works > 60 hours
def g_m49(X):
  return (X['COW'] == 1.0) & (X['WKHP'] > 60)

# private non-profit and works < 10
def g_m50(X):
  return (X['COW'] == 1.0) & (X['WKHP'] <= 10)

# private non-profit and works [11, 20] hours
def g_m51(X):
  return (X['COW'] == 1.0) & (X['WKHP'] > 10) & (X['WKHP'] <= 20)

# private non-profit and works [31, 60] hours
def g_m52(X):
  return (X['COW'] == 1.0) & (X['WKHP'] > 30) & (X['WKHP'] <= 60)

# private non-profit and works > 60 hours
def g_m53(X):
  return (X['COW'] == 1.0) & (X['WKHP'] > 60)

In [118]:
# manager
def g_m54(X):
  return X['OCCP'] <= 440

# business
def g_m55(X):
  return (X['OCCP'] >= 500) & (X['OCCP'] <= 750)

# finance
def g_m56(X):
  return (X['OCCP'] >= 800) & (X['OCCP'] <= 960)

# communication
def g_m57(X):
  return (X['OCCP'] >= 1005) & (X['OCCP'] <= 1240)

# engineering
def g_m58(X):
  return (X['OCCP'] >= 1305) & (X['OCCP'] <= 1560)

# science
def g_m59(X):
  return (X['OCCP'] >= 1600) & (X['OCCP'] <= 1980)

# cms
def g_m60(X):
  return (X['OCCP'] >= 2001) & (X['OCCP'] <= 2060)

# legal
def g_m61(X):
  return (X['OCCP'] >= 2105) & (X['OCCP'] <= 2180)

# education
def g_m62(X):
  return (X['OCCP'] >= 2205) & (X['OCCP'] <= 2555)

# ENT
def g_m63(X):
  return (X['OCCP'] >= 2600) & (X['OCCP'] <= 2920)

# medicine
def g_m63(X):
  return (X['OCCP'] >= 3000) & (X['OCCP'] <= 3550)

# hls
def g_m63(X):
  return (X['OCCP'] >= 3601) & (X['OCCP'] <= 3655)

# prt
def g_m64(X):
  return (X['OCCP'] >= 3700) & (X['OCCP'] <= 3960)

# cln
def g_m65(X):
  return (X['OCCP'] >= 4000) & (X['OCCP'] <= 4160)

# prs
def g_m66(X):
  return (X['OCCP'] >= 4200) & (X['OCCP'] <= 4255)

# sal
def g_m67(X):
  return (X['OCCP'] >= 4330) & (X['OCCP'] <= 4655)

# off
def g_m68(X):
  return (X['OCCP'] >= 4700) & (X['OCCP'] <= 4965)

# fff
def g_m69(X):
  return (X['OCCP'] >= 5000) & (X['OCCP'] <= 5940)

# con
def g_m70(X):
  return (X['OCCP'] >= 6005) & (X['OCCP'] <= 6130)

# ext
def g_m71(X):
  return (X['OCCP'] >= 6200) & (X['OCCP'] <= 6765)

# rpr
def g_m72(X):
  return (X['OCCP'] >= 6800) & (X['OCCP'] <= 6950)

# prd
def g_m73(X):
  return (X['OCCP'] >= 7000) & (X['OCCP'] <= 7640)

# trn 
def g_m74(X):
  return (X['OCCP'] >= 7700) & (X['OCCP'] <= 8990)

# military
def g_m75(X):
  return (X['OCCP'] >= 9005) & (X['OCCP'] <= 9760)

In [123]:
subgroups_original = [g_o1, g_o2, g_o3, g_o4, g_o5, g_o6, g_o7, g_o8, g_o9, g_o10,
                      g_o11, g_o12, g_o13]
subgroups_manual = [g_m1, g_m2, g_m3, g_m4, g_m5, g_m6, g_m7, g_m8, g_m9, g_m10,
                    g_m11, g_m12, g_m13, g_m14, g_m15, g_m16, g_m18, g_m19,
                    g_m20, g_m21, g_m22, g_m23, g_m24, g_m25, g_m27, g_m28,
                    g_m29, g_m30, g_m31, g_m32, g_m33, g_m34, g_m35, g_m36,
                    g_m37, g_m38, g_m39, g_m40, g_m41, g_m42, g_m43, g_m44,
                    g_m45, g_m46, g_m47, g_m48, g_m49, g_m50, g_m51, g_m52,
                    g_m53, g_m54, g_m55, g_m56, g_m57, g_m58, g_m59, g_m60,
                    g_m61, g_m62, g_m63, g_m64, g_m65, g_m66, g_m67, g_m68,
                    g_m69, g_m70, g_m71, g_m72, g_m73, g_m74, g_m75]

manual_groups = subgroups_original + subgroups_manual

# Automated Group Finding

## Epsilon above/below

In [ ]:
# view how different global is from labels
abs_diff = (global_preds - y_train).abs()
abs_diff.describe()

In [80]:
# Train clf to identify rows with big difference

def epsilon_above(epsilon):
    # define 0,1 labels where current predictions OVERESTIMATE by at least epsilon
    binary_labels = (global_preds - y_train) < epsilon

    # define group classifier class
    clf = sk.tree.DecisionTreeClassifier(max_depth = 10, random_state = 42)

    # fit classifier to binary labels
    clf.fit(x_train, binary_labels)

    # define g
    g = clf.predict
    # visualize results
    # pd.DataFrame(g(x_train).astype(int)).describe()
    
    return g

def epsilon_below(epsilon):
    # define 0,1 labels where current predictions OVERESTIMATE by at least epsilon
    binary_labels = (y_train - global_preds) < epsilon

    # define group classifier class
    clf = sk.tree.DecisionTreeClassifier(max_depth = 10, random_state = 42)

    # fit classifier to binary labels
    clf.fit(x_train, binary_labels)

    # define g
    g = clf.predict
    # visualize results
    # pd.DataFrame(g(x_train).astype(int)).describe()
    
    return g

In [ ]:
for i in range(20):
    g = epsilon_below(i*5000)
    train_XGBRegressor(g)
    print(str(i*1000))
    print(check_global_improvement(g,h))

## Targeted Correction

In [14]:
class targeted_correction:
    def __init__(self, clf, value, epsilon):
        self.clf = clf
        self.value = value
        self.epsilon = epsilon

    def __call__(self, X):
        return self.predict(X)
    
    def predict(self, X):
        predictions = self.clf.predict(X)
        return abs(predictions - self.value) < self.epsilon

In [ ]:
# class XGBRegressor_wrap:
#     def __init__(self, clf):
#         self.clf = clf
#     def __call__(self, X):
        

In [15]:
clf = sk.tree.DecisionTreeRegressor(max_depth = 7, random_state = 42)
clf.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=7, random_state=42)

In [ ]:
# data exploration
for i in range(100):
    g = targeted_correction(clf, i*1000, 5000)
    indices = g(x_train)
    # print(indices)
    if (indices.any()):
        old_RMSE = math.sqrt(mean_squared_error(y_train[indices], global_preds[indices]))
        print(i, old_RMSE)

In [ ]:
for i in range(0, 100):
    g = targeted_correction(clf, i*1000, 10000)
    h = train_XGBRegressor(g)
    if(check_global_improvement(g,h)>0):
        print(i, "found improvement")
        save_pkls(g,h)

In [ ]:
for i in range(0, 1):
    # print("i = {}".format(i))
    for j in range(5, 35):
        v = i*1000
        e = j*400
        print("v = {}, e = {}".format(v, e))
        g = targeted_correction(clf, e, j)
        # h = train_XGBRegressor(g)
        if (check_global_improvement(g,h) > 0):
                print(v,e)
                break;

## Clustering

In [ ]:
class cluster_n:

    def __init__(self, clf, n):
        # define attibutes here. You may add more parameters to the init method (see example below)
        self.clf = clf  
        self.n = n

    # DO NOT CHANGE CALL FUNCTION, FORMAT .predict
    def __call__(self, X):
        return self.predict(X)
    
    def predict(self, X):
        # find instances where cluster is 1
        return self.clf.predict(X) == self.n

    
cluster_clf = sk.cluster.KMeans(n_clusters= 5, random_state = 42, n_init=10)
cluster_clf.fit(x_train)
g = cluster_n(cluster_clf, 1)

# visualize results
g(x_train)

In [64]:
def save_cluster_pkls(g, h, i=""):
    # save group function to g.pkl
    g_path = "cluster_pkls/g{}.pkl".format(i)
    h_path = "cluster_pkls/h{}.pkl".format(i)
    
    with open(g_path, 'wb') as file:
        pkl.dump(g, file)

    # save hypothesis function to h.pkl
    with open(h_path, 'wb') as file:
        pkl.dump(h, file)

In [82]:
n_list = [5, 10, 20]

for n in n_list:
    print("\n starting n=" + str(n) + "\n")
    cluster_clf = sk.cluster.KMeans(n_clusters=n, random_state = 42, n_init=10)
    cluster_clf.fit(x_train)
    
    for i in range(0, n):
        g = cluster_n(cluster_clf, i)
        h = train_XGBRegressor(g)
        if (check_local_improvement(g,h) > 0):
            print(n, i, check_global_improvement(g,h), g(x_train).sum())
            save_cluster_pkls(g,h,i)
            
        


 starting n=5

improvement: -2200.9091412623093
improvement: -2765.3381242098367
improvement: -1596.7487944853165
improvement: -2015.033648657005
improvement: -2142.5632406583936

 starting n=10

improvement: -2119.7239629956894
improvement: -2239.921114694691
improvement: -2866.4242477043244
improvement: -1857.3110728773863
improvement: -3318.8520944460943
improvement: -2584.843670247239
improvement: -2172.1347421115497
improvement: -1849.9188912572445
improvement: -2875.379983131108
improvement: -2831.8008721450733

 starting n=20

improvement: -2150.8133293458886
improvement: -3366.538629356568
improvement: -3433.5876706071977
improvement: -2005.497083879718
improvement: -2933.1650537572496
improvement: -1978.8884914322534
improvement: -3238.0788613779005
improvement: -2626.298311095883
improvement: -3032.0652933981146
improvement: -3197.203084537363
improvement: -1871.7640301434349
improvement: -1954.316042394028
improvement: -3167.905044728068
improvement: -3180.1074839453213
imp

## Random Subset

In [ ]:
def set_random_state(state = 42):
    np.random.seed(state)
set_random_state()

In [96]:
# reset random state to make deterministic
def get_random_g(state=42):
  set_random_state(state)
  weight = .3
  random_binary_labels = np.random.rand(len(x_train)) < weight
  clf = sk.tree.DecisionTreeClassifier(max_depth = 10, random_state = 42)
  clf.fit(x_train, random_binary_labels)
  g = clf.predict
  g(x_train)
  return g

In [ ]:
improvements = []
for i in range(1000):
    g = get_random_g(i)
    h = train_basic_h(g)
    imp = check_global_improvement(g,h)
    if(imp>0):
        print(i)
        improvements.append((i, imp))
        # save_pkls(g,h)


sorted_list = sorted(improvements, key=lambda x: x[1])
sorted_list.reverse()
print(sorted_list)

In [ ]:
g = get_random_g(985)
h = train_basic_h(g)
imp = check_global_improvement(g,h)
save_pkls(g,h)

# Gradient Boosting

In [107]:
class base_model:
    def __init__(self, clf):
        self.clf = clf

    def fit(self, X, y):
        return self.clf
    
    def predict(self, X):
        return self.clf.predict(X)

In [135]:
# set up base
clf = sk.tree.DecisionTreeRegressor(max_depth = 10, random_state = 42)
clf.fit(x_train.values, y_train.ravel())
initial_model_gb = base_model(clf)

In [153]:
g = g = manual_groups[42]
indices = g(x_train_subset)
clf = sk.ensemble.GradientBoostingRegressor(max_depth = 3, n_estimators = 100, random_state = 42, init = initial_model_gb)
# calibrate clf with 100 bins on group indices
clf.fit(x_train_subset[indices].values, y_train_subset[indices].ravel())
# define h for architecture
h = clf.predict
print(check_global_improvement(g,h))
save_pkls(g,h)

improvement: 401.78792424308995

 IMPROVEMENT 
 IMPROVEMENT 
 IMPROVEMENT 

401.78792424308995


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [151]:
# define some 'good' clf.
for i in range(len(manual_groups)):
    g = manual_groups[i]
    indices = g(x_train_subset)
    # gradient boost base model specific to g
    if(indices.any()):
        clf = sk.ensemble.GradientBoostingRegressor(max_depth = 3, n_estimators = 100, random_state = 42, init = initial_model_gb)
        # calibrate clf with 100 bins on group indices
        clf.fit(x_train_subset[indices].values, y_train_subset[indices].ravel())
        # define h for architecture
        h = clf.predict
        print(i, check_global_improvement(g,h))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


improvement: -2297.634791948696
0 -2297.634791948696


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


improvement: -1543.7263588128772
1 -1543.7263588128772


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


...

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


improvement: -1696.7215347483034
83 -1696.7215347483034


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


improvement: -1493.7249920644572
84 -1493.7249920644572
improvement: -1870.190246532191
85 -1870.190246532191


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
